This notebook processes IPCC climate predictions for temperature and precipitation rates from latitude-longitude data layout in arrays to averages over each 3-digit zipcode area.

In [1]:
import numpy as np
import pandas as pd

In [ ]:
# Load data frame that contains all economic and environmental data tabulated at 3-digit zipcode level

master_df = pd.read_pickle('master_econ_env_data_zcta.pkl')
rcp_pred_df = master_df.copy()

In [3]:
# Load file with temperatures, temperature trends, precipitation rates, precipitation trends
# predicted for 2027-2030 using IPCC RCP4_5 ("best case scenario") climate model

data = np.load('RCP4_5_precip_temp.npz')
data.files

['ts_trend', 'ts_mean', 'pr_trend', 'pr_mean', 'lat', 'lon']

In [32]:
# Load data and shift longitude by 180 degrees to align with longitudinal convention

ts_trend = data['ts_trend']
ts_mean = data['ts_mean']
pr_trend = data['pr_trend']
pr_mean = data['pr_mean']
lat_v = data['lat']
lon_v = data['lon']
lon_v_adj = np.zeros((len(lon_v),))
for i in range(len(lon_v_adj)):
    if lon_v[i]<=180:
        lon_v_adj[i]=lon_v[i]
    else:
        lon_v_adj[i]=lon_v[i]-360
Lat,Lon = np.meshgrid(lat_v,lon_v_adj)

In [ ]:
# Now for each 3-digit zipcode, either 
# (1) average data within the zipcode radius (approximated) using "as-crow-flies" lat/lon distance or
# (2) if no data points within the zipcode radius, find the nearerst data point and record distance

rcp_pred_df['RCP4_5_precip_mean']=np.nan
rcp_pred_df['RCP4_5_precip_trend']=np.nan
rcp_pred_df['RCP4_5_temp_mean']=np.nan
rcp_pred_df['RCP4_5_temp_trend']=np.nan

for j in range(len(rcp_pred_df)):
    lat1 = rcp_pred_df['INTPTLAT'][j]
    lon1 = rcp_pred_df['INTPTLONG'][j]
    dist = np.sqrt(((Lat-lat1)*111.)**2+(111*(Lon-lon1))**2)
    
    ind1 = np.where(dist<=rcp_pred_df.loc[j,'Radius'])
    if len(ind1[0])>0:
        rcp_pred_df.loc[j,'RCP4_5_precip_mean']=np.nanmean(pr_mean[ind1[1],
                                                                    ind1[0]])
        rcp_pred_df.loc[j,'RCP4_5_precip_trend']=np.nanmean(pr_trend[ind1[1],
                                                                     ind1[0]])
        
        rcp_pred_df.loc[j,'RCP4_5_temp_mean']=np.nanmean(ts_mean[ind1[1],
                                                                    ind1[0]])
        rcp_pred_df.loc[j,'RCP4_5_temp_trend']=np.nanmean(ts_trend[ind1[1],
                                                                     ind1[0]])
    else:
        ii = np.where(dist==np.nanmin(dist))
        rcp_pred_df.loc[j,'RCP4_5_precip_mean']=pr_mean[ii[1],ii[0]]
        rcp_pred_df.loc[j,'RCP4_5_precip_trend']=pr_trend[ii[1],ii[0]]
        rcp_pred_df.loc[j,'RCP4_5_temp_mean']=ts_mean[ii[1],ii[0]]
        rcp_pred_df.loc[j,'RCP4_5_temp_trend']=ts_trend[ii[1],ii[0]]

In [45]:
# Repeat the same procedure with temperatures, temperature trends, precipitation rates, precipitation trends
# predicted for 2027-2030 using IPCC RCP8_5 ("intermediate case scenario") climate model

data = np.load('RCP8_5_precip_temp.npz')
ts_trend = data['ts_trend']
ts_mean = data['ts_mean']
pr_trend = data['pr_trend']
pr_mean = data['pr_mean']
lat_v = data['lat']
lon_v = data['lon']
lon_v_adj = np.zeros((len(lon_v),))
for i in range(len(lon_v_adj)):
    if lon_v[i]<=180:
        lon_v_adj[i]=lon_v[i]
    else:
        lon_v_adj[i]=lon_v[i]-360
Lat,Lon = np.meshgrid(lat_v,lon_v_adj)

rcp_pred_df['RCP8_5_precip_mean']=np.nan
rcp_pred_df['RCP8_5_precip_trend']=np.nan
rcp_pred_df['RCP8_5_temp_mean']=np.nan
rcp_pred_df['RCP8_5_temp_trend']=np.nan

for j in range(len(rcp_pred_df)):
    lat1 = rcp_pred_df['INTPTLAT'][j]
    lon1 = rcp_pred_df['INTPTLONG'][j]
    dist = np.sqrt(((Lat-lat1)*111.)**2+(111*(Lon-lon1))**2)
    
    ind1 = np.where(dist<=rcp_pred_df.loc[j,'Radius'])
    if len(ind1[0])>0:
        rcp_pred_df.loc[j,'RCP8_5_precip_mean']=np.nanmean(pr_mean[ind1[1],
                                                                 ind1[0]])
        rcp_pred_df.loc[j,'RCP8_5_precip_trend']=np.nanmean(pr_trend[ind1[1],
                                                                     ind1[0]])
        
        rcp_pred_df.loc[j,'RCP8_5_temp_mean']=np.nanmean(ts_mean[ind1[1],
                                                                    ind1[0]])
        rcp_pred_df.loc[j,'RCP8_5_temp_trend']=np.nanmean(ts_trend[ind1[1],
                                                                     ind1[0]])
    else:
        ii = np.where(dist==np.nanmin(dist))
        rcp_pred_df.loc[j,'RCP8_5_precip_mean']=pr_mean[ii[1],ii[0]]
        rcp_pred_df.loc[j,'RCP8_5_precip_trend']=pr_trend[ii[1],ii[0]]
        rcp_pred_df.loc[j,'RCP8_5_temp_mean']=ts_mean[ii[1],ii[0]]
        rcp_pred_df.loc[j,'RCP8_5_temp_trend']=ts_trend[ii[1],ii[0]]

In [50]:
# Pickle the dataframe so that it can be used later (Predicting_Future_HPI.ipynb)
rcp_pred_df.to_pickle('ipcc_master_df.pkl')